# Comparing the features extracted from Python Wrapper V.S. C++

# Part 1: Extract Features with Caffe Python wrapper

The following code is modified from https://github.com/BVLC/caffe/blob/master/examples/00-classification.ipynb

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

import os
if not os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print("Downloading pre-trained CaffeNet model...")
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet
    
caffe.set_mode_cpu()
net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
                caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))#
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

net.blobs['data'].reshape(1,3,227,227)
net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(caffe_root + 'examples/images/cat.jpg'))
out = net.forward()
print("First 5-dim of predicted probability is \n #{}.".format(out['prob'][0][0:5]))

# load labels
imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
try:
    labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
except:
    !../data/ilsvrc12/get_ilsvrc_aux.sh
    labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

# sort top k predictions from softmax output
top_k = net.blobs['prob'].data[0].flatten().argsort()[-1:-6:-1]
print "\n Top 5 classes: \n", labels[top_k]

First 5-dim of predicted probability is 
 #[  1.19005195e-09   1.75350069e-05   2.98270955e-08   1.97281977e-08
   6.59751453e-09].

 Top 5 classes: 
['n02123045 tabby, tabby cat' 'n02123159 tiger cat'
 'n02124075 Egyptian cat' 'n02119022 red fox, Vulpes vulpes'
 'n02127052 lynx, catamount']


## Part 2: Read features extracted from C++

I extracted feature from the same image (cat.jpg) following the instruction 
http://caffe.berkeleyvision.org/gathered/examples/feature_extraction.html
with minor modifications:

(1)(2) are not necessary. I did it just to make sure the features extracted come from "cat.jpg" and not other images. <br>
(1) In the file "caffe/examples/_temp/file_list.txt", I remove the other two images and keep only cat.jpg.  <br>
(2) In the file "caffe/examples/_temp/imagenet_val.prototxt", change batch_size to 1 <br>
(3) Run: ./build/tools/extract_features.bin models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel  examples/_temp/imagenet_val.prototxt prob examples/_temp/features 1 lmdb GPU <br>
So that probability-layer is used instead of fc7, only 1 feature is extracted, is saved in lmdb format, and use GPU. 

In [2]:
## This code is modified from 
## http://stackoverflow.com/questions/33117607/caffe-reading-lmdb-from-python

import caffe
import lmdb
import numpy as np
from caffe.proto import caffe_pb2
caffe_root = '../'
def save2txt(db_name):
    img_db = lmdb.open(db_name)
    txn = img_db.begin()
    cursor = txn.cursor()
    cursor.iternext()
    
    count = 0
    train={}
    
    datum = caffe_pb2.Datum()
    count = 0
    train={}
    for key, value in cursor:
        datum.ParseFromString(value)
        data = caffe.io.datum_to_array(datum)
        data = np.reshape(data, (1, np.product(data.shape)))[0]
        train[count]=data
        count +=1    
    return train


In [3]:
prob_feature=save2txt('/home/ncchen/caffe/examples/_temp/features/')
print "The first 5-dim of (probability) feature vector: \n", prob_feature[0][0:5]

The first 5-dim of (probability) feature vector: 
[  7.86413157e-09   2.60096385e-05   4.55995348e-08   5.70442324e-08
   2.98107672e-08]


In [4]:
top_k = prob_feature[0].flatten().argsort()[-1:-6:-1]
print "\n Top 5 classes: \n", labels[top_k]


 Top 5 classes: 
['n02123045 tabby, tabby cat' 'n02124075 Egyptian cat'
 'n02123159 tiger cat' 'n02127052 lynx, catamount'
 'n02119789 kit fox, Vulpes macrotis']
